# Step One: Setup

### Integrating Environmental Factors into Storm Water Structure Point Layer

**Main Layer**: **Storm Water Structure Point Layer**

#### Environmental Factors:
1. **Distance to Stream**
2. **Distance to Roads**
3. **Digital Elevation Model (DEM)**


### 1. Distance to Streams

###### Source: https://www.nconemap.gov/datasets/e6b5694f81ed49c6a0f3600540f49878_0/explore?location=35.315422%2C-80.715475%2C14.61

In [23]:
streamPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\01_dataTransformation\01_streamDistance\streamClipped.shp"

### 2. Distance to Roads

###### Source: https://connect.ncdot.gov/resources/gis/Pages/GIS-Data-Layers.aspx

In [24]:
roadPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\01_dataTransformation\02_roadsDistance\roadsClipped.shp"

### 3. Mecklenburg DEM

###### Source: https://sdd.nc.gov/DownloadFiles.aspx?path=DEMMosaicsbyCounty/DEM50_CountywideRasters

In [54]:
demPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\01_dataTransformation\03_meckDEM\Mecklenburg_Ground_50ft.tif"

### 4. Storm Water Structures

In [26]:
stormPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\00_theData\inlet.csv"

# Step Two: Coding

### Import Statements

In [67]:
import pandas as pd
import geopandas as gpd
import time
from shapely.geometry import Point
from rasterstats import point_query
import osgeo
from osgeo import gdal
from shapely.ops import nearest_points
import fiona

### Structures CSV into DataFrame

In [28]:
stormpoints_df = pd.read_csv(stormPath)
stormpoints_gdf = gpd.GeoDataFrame(
    stormpoints_df,
    geometry=gpd.points_from_xy(stormpoints_df['Longitude'], stormpoints_df['Latitude']),
    crs="EPSG:2264"
)

### Loading Road GDF

In [ ]:
roads_gdf = gpd.read_file(roadPath)

target_crs = "EPSG:2264"
stormpoints_gdf = stormpoints_gdf.to_crs(target_crs)
print(stormpoints_gdf.crs)
roads_gdf = roads_gdf.to_crs(target_crs)
# roads_gdf was originally loaded as 6360 but GeoPandas can't handle Compound CRS
print(roads_gdf.crs)



### Distance to Nearest Road

In [30]:
def min_distance_to_roads(stormpoint, roads):
    return roads.distance(stormpoint).min()

stormpoints_gdf['nearest_road'] = stormpoints_gdf['geometry'].apply(
    lambda point: min_distance_to_roads(point, roads_gdf)
)

### Loading Streams GDF

In [31]:
streams_gdf = gpd.read_file(streamPath)
streams_gdf = streams_gdf.to_crs(target_crs)

### Distance to Nearest Stream

In [32]:
def min_distance_to_streams(stormpoint,streams):
    return streams.distance(stormpoint).min()

stormpoints_gdf['nearest_stream'] = stormpoints_gdf['geometry'].apply(
    lambda point: min_distance_to_streams(point, streams_gdf)
)
    

### DEM


In [ ]:
elevations = point_query(stormpoints_gdf, demPath, property_name='elevation')
stormpoints_gdf['elevation'] = [elev[0] if elev else None for elev in elevations]
print(f'calculating time: {time.time()-start:.2f} seconds')
start=time.time()

### DEM to Vector

In [ ]:
contour_shp = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\03_outputTesting\contours.shp"
#Generate Contours
# gdal.ContourGenerate(demPath, contour_shp, interval=10, field_name="elevation")

import subprocess

subprocess.run([
    "gdal_contour",
    "-i", "10",                   # Contour interval
    "-a", "elevation",             # Attribute field for elevation
    "-f", "ESRI Shapefile",        # Format as Shapefile
    demPath,
    contour_shp
])

### Finding Elevation

In [ ]:
contours_gdf = gpd.read_file(contour_shp)

# Ensure both layers are in the same CRS
if stormpoints_gdf.crs != contours_gdf.crs:
    stormpoints_gdf = stormpoints_gdf.to_crs(contours_gdf.crs)

# Define function to find the nearest contour elevation for each point
def find_nearest_elevation(point, contours):
    nearest_geom = contours.geometry.unary_union  # Combine all contours into one geometry for nearest search
    nearest_line = nearest_points(point, nearest_geom)[1]
    nearest_elevation = contours[contours.geometry == nearest_line]['elevation'].values[0]
    return nearest_elevation

# Apply this function to each point in stormpoints_gdf
stormpoints_gdf['elevation'] = stormpoints_gdf['geometry'].apply(
    lambda point: find_nearest_elevation(point, contours_gdf)
)


# Step Three: Export


### Results to CSV

In [35]:
outputPath = r"C:\Users\rcarte64\Downloads\Task3_DeepPipeScript\03_outputTesting\roads_streams_DEM_Test.csv"

In [36]:
output_df = stormpoints_gdf[['OBJECTID', 'Latitude', 'Longitude', 'nearest_road','nearest_stream','elevation']]
output_df.to_csv(outputPath)